# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [9]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

# generate the scatter plot to make sure that the cities are evenly distributed throughout the globe
plt.scatter(cities["lats"],cities["longs"],marker="o",facecolors="red",edgecolors="black",s=50,alpha=0.75)

TypeError: list indices must be integers or slices, not str

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
# API set up
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="

# Delay the process
record = 1

# Lists to hold data
c_id = []
c_name = []
country = []
lat = []
long = []
t_max = []
humidity = []
cloudiness = []
w_speed = []

# Create a file to store Open WeatherMap API call info and store as a .csv
owmlog = open('open_weathermap_log.csv', 'w')
owmdata = open('open_weathermap_data.csv', 'w')

# Define header for open_weathermap_data.csv
owmdata.write("c_id,c_name,country,lat,long,temp,humidity,cloudiness,w_speed\n")

# Loop through the list of cities
for city in cities:
    if record == 50:
        record +=1
        time.sleep(60)
    
    query_url = f"{url}{api_key}&q={city}"
    #print(query_url)
    response = requests.get(query_url).json()
    
# Try to querry the pertinent data, creating variables with proper formatting
    try:
        cityID = int(response['id'])
        cityName = response['name']
        countryCode = response['sys']['country']
        lat_val = response['coord']['lat']
        lon_val = response['coord']['lon']
        temp_max = response['main']['temp_max']
        humidity_val = response['main']['humidity']
        cloudiness_val = response['clouds']['all']
        windSpeed = response['wind']['speed']
        
        # Append retreived values to lists above using the variables created above
        c_id.append(cityID)
        c_name.append(cityName)
        country.append(countryCode)
        lat.append(lat_val)
        long.append(lon_val)
        t_max.append(temp_max)
        humidity.append(humidity_val)
        cloudiness.append(cloudiness_val)
        w_speed.append(windSpeed)
       
        # Print cityname & cityID  to mark progress and store it to a print log text file
        print(f"{cityName}:{cityID} was found in the Open Weathermap API\n")
        owmlog.write(f"{cityName}:{cityID} was found in the Open Weathermap API\n")
        
        # Store the data as a .csv file
        owmdata.write(f"{cityID},{cityName},{countryCode},{lat_val},{lon_val},\
            {t_max},{humidity_val},{cloudiness_val},{windSpeed}\n")
        
    # If the data is not in Open WeatherMap API and it creates a Null
    except:
        
        print(f"{city} was not found in the Open Weathermap API\n")
        owmlog.write(f"{city} was not found in the Open Weathermap API\n")
        record +=1
        
        #Add these cities to the lists
        c_id.append(None)
        c_name.append(None)
        country.append(None)
        lat.append(None)
        long.append(None)
        t_max.append(None)
        humidity.append(None)
        cloudiness.append(None)
        w_speed.append(None)
        pass
    
# Close the file handle after completion of the loop
owmlog.close()
owmdata.close()

Castro:3466704 was found in the Open Weathermap API

Leningradskiy:2123814 was found in the Open Weathermap API

Hermanus:3366880 was found in the Open Weathermap API

Rikitea:4030556 was found in the Open Weathermap API

Port Alfred:964432 was found in the Open Weathermap API

Mataura:6201424 was found in the Open Weathermap API

Bethel:5282297 was found in the Open Weathermap API

Qaanaaq:3831208 was found in the Open Weathermap API

Khatanga:2022572 was found in the Open Weathermap API

Bonavista:5905393 was found in the Open Weathermap API

Province of Mantua:3174050 was found in the Open Weathermap API

taolanaro was not found in the Open Weathermap API

Naryan-Mar:523392 was found in the Open Weathermap API

mys shmidta was not found in the Open Weathermap API

Najrān:103630 was found in the Open Weathermap API

najran was not found in the Open Weathermap API

Jiuquan:1279945 was found in the Open Weathermap API

Meulaboh:1214488 was found in the Open Weathermap API

Saskylakh:20

Saint George:5546220 was found in the Open Weathermap API

Alekseyevsk:2022083 was found in the Open Weathermap API

Son La:1567681 was found in the Open Weathermap API

Masallātah:2215163 was found in the Open Weathermap API

masallatah was not found in the Open Weathermap API

Saint-Philippe:935215 was found in the Open Weathermap API

Tasiilaq:3424607 was found in the Open Weathermap API

Rumonge:423549 was found in the Open Weathermap API

Chumphon:1153555 was found in the Open Weathermap API

Kenai:5866063 was found in the Open Weathermap API

Pemagatsel:1337379 was found in the Open Weathermap API

Vila do Maio:3374120 was found in the Open Weathermap API

Luanda:2240449 was found in the Open Weathermap API

Hervey Bay:2146219 was found in the Open Weathermap API

Te Anau:2181625 was found in the Open Weathermap API

Tessalit:2449893 was found in the Open Weathermap API

Altamira:3407882 was found in the Open Weathermap API

Aklavik:5882953 was found in the Open Weathermap API

T

Mackay:2159220 was found in the Open Weathermap API

Port Hardy:6111862 was found in the Open Weathermap API

tunxi was not found in the Open Weathermap API

Babstovo:2027277 was found in the Open Weathermap API

Geraldton:2070998 was found in the Open Weathermap API

Marsá Maţrūḩ:352733 was found in the Open Weathermap API

marsa matruh was not found in the Open Weathermap API

Yangi Marg`ilon:1513491 was found in the Open Weathermap API

Seymchan:2121373 was found in the Open Weathermap API

Ponta Delgada:3372783 was found in the Open Weathermap API

Padang:1633419 was found in the Open Weathermap API

Pangnirtung:6096551 was found in the Open Weathermap API

galgani was not found in the Open Weathermap API

Vila Velha:3445026 was found in the Open Weathermap API

Vanavara:2013727 was found in the Open Weathermap API

tumannyy was not found in the Open Weathermap API

Gaoyou:1810309 was found in the Open Weathermap API

Northam:2641434 was found in the Open Weathermap API

alotau was

Heihe:2036973 was found in the Open Weathermap API

Ubari:2219235 was found in the Open Weathermap API

tsihombe was not found in the Open Weathermap API

Kaeo:2189343 was found in the Open Weathermap API

Rønne:2614553 was found in the Open Weathermap API

dolbeau was not found in the Open Weathermap API

Umm Hājar:2426370 was found in the Open Weathermap API

oum hadjer was not found in the Open Weathermap API

Morón:3430545 was found in the Open Weathermap API

Ulladulla:2145554 was found in the Open Weathermap API

South Valley:5492450 was found in the Open Weathermap API

Chernyshevskiy:2025456 was found in the Open Weathermap API

Eagle Pass:5520677 was found in the Open Weathermap API

Nísia Floresta:3393922 was found in the Open Weathermap API

Avilés:3129135 was found in the Open Weathermap API

Show Low:5314245 was found in the Open Weathermap API

Pingliang:1798760 was found in the Open Weathermap API

Pápa:3046686 was found in the Open Weathermap API

Quang Ngai:1568770 was

Olavarría:3842670 was found in the Open Weathermap API

Shreveport:4341513 was found in the Open Weathermap API

Urubichá:3901969 was found in the Open Weathermap API

Chāndūr Bāzār:1274674 was found in the Open Weathermap API

chandur bazar was not found in the Open Weathermap API

Mineiros:3457147 was found in the Open Weathermap API

Santa Maria:3450083 was found in the Open Weathermap API

Nemuro:2128975 was found in the Open Weathermap API

Kapuvár:3050594 was found in the Open Weathermap API

Dudinka:1507116 was found in the Open Weathermap API

Brae:2654970 was found in the Open Weathermap API

Pontianak:1630789 was found in the Open Weathermap API

Olafsvik:3414079 was found in the Open Weathermap API

Ardakān:143073 was found in the Open Weathermap API

ardakan was not found in the Open Weathermap API

Arman':2127060 was found in the Open Weathermap API

Katrineholm:2701223 was found in the Open Weathermap API

Matara:1235846 was found in the Open Weathermap API

Riyadh:108410

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [6]:
cities_df = pd.read_csv('open_weathermap_data.csv')
cities_df.head()

ParserError: Error tokenizing data. C error: Expected 9 fields in line 3, saw 10


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(cities_df["lat"], cities_df["temp_max"], marker = "^")
plt.title("Latitude vs. Max Temperature")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature")

plt.savefig("Pics/Lat_v_max_temp.png")
plt.show()

print("Conclusion")

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(cities_df["lat"], cities_df["humidity"], marker = "^")
plt.title("Latitude vs. Humidity")
plt.xlabel("Latitude")
plt.ylabel("Humidity")

plt.savefig("Pics/Lat_v_Humidity.png")
plt.show()

print("Conclusion")

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(cities_df["lat"], cities_df["cloudiness"], marker = "^")
plt.title("Latitude vs. Cloudiness")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")

plt.savefig("Pics/Lat_v_Cloudiness.png")
plt.show()

print("Conclusion")

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(cities_df["lat"], cities_df["w_speed"], marker = "^")
plt.title("Latitude vs. Wind Speed")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")

plt.savefig("Pics/Lat_v_w_speed.png")
plt.show()

print("Conclusion")

## Linear Regression

In [ ]:
#Divide the cities into Northern and Southern Hemisphere DataFrames
northern_df = cities_df.loc[cities_df["lat"].astype(float) > 0]
southern_df = cities_df.loc[cities_df["lat"].astype(float) < 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Run Linear Regression Analysis
x_values = northern_df["lat"]
y_values = northern_df["temp_max"]

# Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

# Plot the regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

# Title & Axis labels
plt.title("Northern Hemisphere - Temperature (F) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_Lat_v_Max_temp.png")
plt.show()

print("conclusion")

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Analysis
x_values = southern_df["lat"]
y_values = southern_df["temp_max"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Northern Hemisphere - Temperature (F) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_Lat_v_temp_max.png")
plt.show()

print("conclusion")

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Analysis
x_values = northern_df["lat"]
y_values = northern_df["humidity"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Northern Hemisphere - Humidity (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_Hum_vs_Lat.png")
plt.show()

print("conclusion")

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Analysis
x_values = southern_df["lat"]
y_values = southern_df["humidity"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Southern Hemisphere - Humidity (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/South_Hem_Hum_vs_Lat.png")
plt.show()

print("conclusion")

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Analysis
x_values = northern_df["lat"]
y_values = northern_df["clouds"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Northern Hemisphere - Cloudiness (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_Cloudi_vs_Lat.png")
plt.show()

print("Conclusion")

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:

#Run Linear Regression Data
x_values = southern_df["lat"]
y_values = southern_df["clouds"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Southern Hemisphere - Cloudiness (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/South_Hem_Cloudi_vs_Lat.png")
plt.show()

print("Conclusion")

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Data
x_values = northern_df["lat"]
y_values = northern_df["wind"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Northern Hemisphere - Wind Speed (mph) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_W_Speed_vs_Lat.png")
plt.show()

print("Conclusion.")

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Data
x_values = southern_df["lat"]
y_values = southern_df["wind"]

#Plot scatter plot
plt.scatter(x_values,y_values)


(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))



#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Southern Hemisphere - Wind Speed (mph) vs. Latitude (mph)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/South_Hem_W_Speed_vs_Lat.png")
plt.show()

print("Conclusion")